# Proyecto Capstone

<P>Agrupación y segmentación de vecindarios de la ciudad de Toronto, en Canadá.</P>

<h4><p>Realizado por: Cristian Parada</p></h4>

## Parte 1. Crear el dataframe con los datos del código postal:

In [4]:
#Primero procedemos a importar todas las librerías necesarias para el ejercicio:

!pip install geopy
import numpy as np 

import pandas as pd 
from pandas import DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json  
from geopy.geocoders import Nominatim 

import requests 

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium 
import urllib3
print('Todas las librerias importadas!')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 5.6 MB/s eta 0:00:00


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing th

Todas las librerias importadas!


In [26]:
# Extraemos el archivo de Wikipedia:

WikiTables = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1008658788', header=0)
WikiTables

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 5           M6A        North York   
 6           M7A  Downtown Toronto   
 7           M8A      Not assigned   
 8           M9A         Etobicoke   
 9           M1B       Scarborough   
 10          M2B      Not assigned   
 11          M3B        North York   
 12          M4B         East York   
 13          M5B  Downtown Toronto   
 14          M6B        North York   
 15          M7B      Not assigned   
 16          M8B      Not assigned   
 17          M9B         Etobicoke   
 18          M1C       Scarborough   
 19          M2C      Not assigned   
 20          M3C        North York   
 21          M4C         East York   
 22          M5C  Downtown Toronto   
 23          M6C              York   
 24          M7C      Not assigned   
 25         

In [8]:
# Comprobemos si en dicho archivo se encuentra solamente la tabla que buscamos o hay más:

print(F'Número de tablas: {len(WikiTables)}')
print('Como podemos ver, en el archivo HTML hay 3 tablas. Debemos seleccionar la que queremos tratar:')

Número de tablas: 3
Como podemos ver, en el archivo HTML hay 3 tablas. Debemos seleccionar la que queremos tratar:


In [9]:
# Primero veamos las primeras líneas de cada tabla para poder identificarla:

for i in range(len(WikiTables)):
    print(WikiTables[i].head(2))

  Postal Code       Borough Neighbourhood
0         M1A  Not assigned  Not assigned
1         M2A  Not assigned  Not assigned
  .mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output .navbar-brackets::before{margin-right:-0.125em;content:"[ "}.mw-parser-output .navbar-brackets::after{margin-left:-0.125em;content:" ]"}.mw-parser-output .navbar li{word-spacing:-0.125em}.mw-parser-output .navbar a>span,.mw-parser-output .navbar a>abbr{text-decoration:inherit}.mw-parser-output .navbar-mini abbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output .navbar-ct-full{font-size:114%;margin:0 7em}.mw-parser-output .navbar-ct-mini{font-size:114%;margin:0 4em}vteCanadian postal codes  \
0                                   

In [10]:
# Vemos que la tabla de los códigos postales que nos interesa es la primera, vamos a seleccionarla e imprimirla en un dataframe:

CPT = WikiTables[0]
CPT

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [ ]:
# Perfecto! Ahora tenemos que formatear la tabla en base a las directrices que nos pide el ejercicio:

# 1º eliminamos las celdas que no tienen un municipio asignado

CPT.replace("Not assigned", np.nan, inplace=True) # Para facilitar el trabajo remplazamos el valor 'Not assigned' a un marcador 'NaN'
CPT.head()

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [13]:
#Buscando la data perdida en nuestro set de datos.
missing_data = CPT.isnull()
missing_data
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print("")

Postal Code
False    180
Name: Postal Code, dtype: int64

Borough
False    103
True      77
Name: Borough, dtype: int64

Neighbourhood
False    103
True      77
Name: Neighbourhood, dtype: int64



In [14]:
# Vemos que los valores NaN coinciden para Borough y Neighbourhood, por tanto podemos eliminar todas las filas en las que haya NaN:

CPT.dropna(axis=0, inplace=True)
CPT

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [15]:
# Por cuestión de estilo, vamos a reiniciar los índices:

CPT.reset_index(drop=True, inplace=True)
CPT


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [19]:
# Veamos el numero de filas y columnas que quedan una vez tratado el dataframe:
CPT.shape
print(CPT.shape)
print(Vemos que el dataframe restante tiene 103 filas y 3 columnas')

(103, 3)
Vemos que el dataframe restante tiene 103 filas y 3 columnas


## Parte 2. Añadir datos geoespaciales

In [68]:
#Realizaremos la toma de coordenadas mediante la traida de archivo con las coordenadas a nuestro proyecto.

Coordenadas = pd.read_csv("Geospatial_Coordinates.csv")
Coordenadas

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [69]:
# Podemos ver que este dataframe está ordenado alfabéticamente en base al código postal.
# También, comprobando su shape, podemos ver si nuestro cálculo de la forma del dataframe anterior concuerda con este:

print('Tamaño de la tabla de coordenadas:', Coordenadas.shape)
print('')
print('Vemos que el número de filas es el mismo, por tanto, hay coherencia en los datos.')

Tamaño de la tabla de coordenadas: (103, 3)

Vemos que el número de filas es el mismo, por tanto, hay coherencia en los datos.


In [70]:
# Para añadir los datos geoespaciales a la tabla de Códigos Postales, primero tenemos que hacer algún ajuste, 
# como por ejemplo ordenar alfábeticamente la tabla en base al nombre del código postal:

CPT.sort_values(by=['Postal Code'], inplace = True, ignore_index=True) #ignore_index hace que los índices se reinicien.
CPT

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [71]:
# Una vez ordenada la columna y comprobada su coherencia, vamos a borrar la columna 'Postal Code' en el dataframe de
# las coordenadas y unirlo con el dataframe de los códigos postales:

Coordenadas.drop(['Postal Code'], axis=1, inplace=True) #Eliminamos la columna
Coordenadas

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476
5,43.744734,-79.239476
6,43.727929,-79.262029
7,43.711112,-79.284577
8,43.716316,-79.239476
9,43.692657,-79.264848


In [72]:
#Concatenamos los dos dataframes
TorontoGeoData = pd.concat([CPT, Coordenadas], axis=1) 
TorontoGeoData

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Parte 3. Agrupación de los vecindarios y visualización de los mapas

In [90]:
file_name ='https://cocl.us/Geospatial_data'
Geoconder = pd.read_csv(file_name)

In [92]:
neighborhoods=pd.merge(TorontoGeoData,Geoconder,on="Postal Code")

In [93]:
neighborhoods

,Postal Code,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,43.692657,-79.264848


In [94]:
type(neighborhoods)

pandas.core.frame.DataFrame

In [95]:
print('There are {} uniques Borough.'.format(len(neighborhoods['Borough'].unique())))

There are 10 uniques Borough.


In [96]:
neighborhoods['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [73]:
# Lo primero de todo es entender los datos que tenemos; cuántos barrios, cuántos municipios, etc.

print('El dataframe de Toronto tiene {} municipios (boroughs) y {} barrios (neighborhoods).'.format(
        len(TorontoGeoData['Borough'].unique()),
        TorontoGeoData.shape[0]))

El dataframe de Toronto tiene 10 municipios (boroughs) y 103 barrios (neighborhoods).


In [101]:
# Ahora, a través de la librería geopy vamos a obtener las coordenadas de la ciudad de Toronto:
address = 'TORONTO,CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [111]:
# crear un mapa de Nueva York utilizando los valores de latitud y longitud
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

#Marcador de la ciudad de Toronto
tooltip = "Toronto City"

folium.Marker(
    [43.7001, -79.4163], popup="Toronto",
    tooltip=tooltip

).add_to(map_Toronto)
# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude_x'], neighborhoods['Longitude_x'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)
    
map_Toronto